In [1]:
import os
import pandas as pd 
import numpy as np
import pickle
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.preprocessing.sequence import pad_sequences

import torch
from torch.utils.data import TensorDataset,DataLoader 
from torch.autograd import Variable
from torch.optim import Adam,SGD,RMSprop
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
from torch.nn.init import xavier_uniform_, kaiming_normal_
import time
import gc
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
# sns.set(style='white')
# style.use("fivethirtyeight")

Using TensorFlow backend.


In [2]:
lr = 0.001 # learning rate
batch_size = 500
weight_decay = 1e-5
epochs = 10


dropout = 0.4 # replaced by bn layers
cuda_able = True
length_sequence = 90 # 
hidden_size = 256  # 128 -> 256

bidirectional = False
use_pe = False
vocab_size1 = 3412772 + 1 #  creative_id
embed_dim1 = 100

vocab_size2 = 3027360 + 1 # ad_id 
embed_dim2 = 100

vocab_size3 = 57870 + 1 # advertiser_id
embed_dim3 = 100

vocab_size4 = 39057 + 1 # product_id
embed_dim4 = 100

vocab_size5 = 332 + 1 # in_id
embed_dim5 = 100
# seed = 2020
# torch.manual_seed(seed)
use_cuda = torch.cuda.is_available() and cuda_able
print(use_cuda)

early_stopping_patience = 10
save_path = 'bilstm_model.pth'

True


In [3]:
labels = pd.read_csv('train_preliminary/user.csv')
y_gender = labels['gender']
y_age = labels['age']
from sklearn.preprocessing import LabelEncoder
le1 = LabelEncoder() #对性别编码
le1 = le1.fit(y_gender)
y_gender_labels = le1.transform(y_gender)
le2 = LabelEncoder() #对年龄编码
le2 = le2.fit(y_age)
y_age_labels = le2.transform(y_age)


In [4]:
aggregate_features_df = pd.read_csv('aggregate_features_df.csv')
aggregate_features_df.head(5)

,user_id,creative_id_nunique,click_times_max,click_times_min,click_times_mean,click_times_std,click_times_count,ad_id_nunique,product_id_nunique,product_category_nunique,advertiser_id_nunique,industry_nunique,time_nunique
0,1,12,2,1,1.076923,0.277350,13,12,6,3,12,9,10
1,2,42,2,1,1.022222,0.149071,45,42,20,3,36,15,28
2,3,30,1,1,1.000000,0.000000,30,30,17,6,28,8,23
3,4,29,1,1,1.000000,0.000000,29,29,18,6,26,10,15
4,5,33,2,1,1.030303,0.174078,33,33,7,4,30,18,26


In [5]:
sta = StandardScaler()
_ = sta.fit_transform(aggregate_features_df.iloc[:,1:].values)

In [6]:
train_df = _[:900000,:]
test_df = _[900000:,:]
train_df[0]

array([-0.3169476 ,  0.21785329, -0.00177705,  0.30150579,  0.48453856,
       -0.21636036, -0.32186687, -0.54664042, -0.65150519, -0.56421382,
       -0.56598012, -0.91885536])

# Text Classification with lstm

In [7]:
def load_sequences():
    with open('creative_id_sequence.pkl','rb') as f: # read creative id
        X_cid_total = pickle.load(f)
    with open('ad_id_sequence.pkl','rb') as f: # ad
        X_aid_total = pickle.load(f)
    with open('advertiser_id_sequence.pkl','rb') as f:
        X_adid_total= pickle.load(f)
    with open('product_id_sequence.pkl','rb') as f:
        X_pid_total= pickle.load(f)
    with open('industry_sequence.pkl','rb') as f:
        X_in_total= pickle.load(f)
    insequence_l = [len(i) if len(i) <= length_sequence else length_sequence for i in X_cid_total]
    return X_cid_total, X_aid_total, X_adid_total,  X_pid_total, X_in_total, insequence_l     ##, X_in_total,X_aid_total X_pc_total,

In [8]:
X_cid_total, X_aid_total, X_adid_total,  X_pid_total, X_in_total,insequence_l  = load_sequences() # ,X_pc_total, X_in_total X_pc_total,

In [9]:
with open('creative_id_embedding_ini.pkl','rb') as em:
    word_embeddings_cid = pickle.load(em)
word_embeddings_cid = word_embeddings_cid
word_embeddings_cid.shape

(3412773, 100)

In [10]:
with open('ad_id_embedding_ini.pkl','rb') as em:
    word_embeddings_aid = pickle.load(em)
word_embeddings_aid = word_embeddings_aid
word_embeddings_aid.shape

(3027361, 100)

In [11]:
with open('advertiser_id_embedding_ini.pkl','rb') as em:
    word_embeddings_adid = pickle.load(em)
word_embeddings_adid = word_embeddings_adid
word_embeddings_adid.shape

(57871, 100)

In [12]:
with open('product_id_embedding_ini.pkl','rb') as em:
    word_embeddings_pid = pickle.load(em)
word_embeddings_pid = word_embeddings_pid
word_embeddings_pid.shape

(39058, 100)

In [13]:
with open('industry_embedding_ini.pkl','rb') as em:
    word_embeddings_in = pickle.load(em)
word_embeddings_in = word_embeddings_in
word_embeddings_in.shape

(333, 100)

In [14]:
# padding

X_cid_total =  pad_sequences(X_cid_total,length_sequence,padding='pre',truncating='pre')
X_aid_total =  pad_sequences(X_aid_total,length_sequence,padding='pre',truncating='pre')
X_adid_total = pad_sequences(X_adid_total,length_sequence,padding='pre',truncating='pre')
X_pid_total =  pad_sequences(X_pid_total,length_sequence,padding='pre',truncating='pre')
X_in_total =   pad_sequences(X_in_total,length_sequence,padding='pre',truncating='pre')

# training validate data
X_cid = X_cid_total[:900000]
X_aid = X_aid_total[:900000]
X_adid = X_adid_total[:900000]
X_pid = X_pid_total[:900000]
X_in = X_in_total[:900000]

train_sequence_length = insequence_l[:900000]

# preditcion data
for_test_cid = X_cid_total[900000:]
for_test_aid = X_aid_total[900000:]
for_test_adid = X_adid_total[900000:]
for_test_pid = X_pid_total[900000:]
for_test_in = X_in_total[900000:]
test_sequence_length = insequence_l[900000:]
del X_cid_total, X_aid_total, X_adid_total, X_pid_total, X_in_total



In [15]:
X_cid = torch.LongTensor(X_cid)
X_aid = torch.LongTensor(X_aid)
X_adid = torch.LongTensor(X_adid)
X_pid = torch.LongTensor(X_pid)
X_in = torch.LongTensor(X_in)
y_gender_labels = torch.LongTensor(y_gender_labels)
y_age_labels = torch.LongTensor(y_age_labels)
count_feature = torch.FloatTensor(train_df)

for_test_cid = torch.LongTensor(for_test_cid)
for_test_aid = torch.LongTensor(for_test_aid)
for_test_adid = torch.LongTensor(for_test_adid)
for_test_pid = torch.LongTensor(for_test_pid)
for_test_in = torch.LongTensor(for_test_in)
for_test_count_feature = torch.FloatTensor(test_df)

In [16]:
# with open('click_times_corpus.pkl','rb') as f:
#     click_time_mask = pickle.load(f)
# click_time_mask = pad_sequences(click_time_mask, length_sequence, padding='pre',truncating='pre')

# train_ctm = click_time_mask[:900000]  # for training
# predict_ctm = click_time_mask[900000:]# for prediction

In [17]:
def get_batch(i,batch_size, X_cid, X_aid, X_adid, X_pid,X_in, count_feature, y_gender_labels, y_age_labels):
    c_batch = X_cid[i * batch_size: (i + 1) * batch_size]
    a_batch = X_aid[i * batch_size: (i + 1) * batch_size]
    ad_batch = X_adid[i * batch_size: (i + 1) * batch_size]
    p_batch = X_pid[i * batch_size: (i + 1) * batch_size]
    in_batch = X_in[i * batch_size: (i + 1) * batch_size]
    count_batch = count_feature[i * batch_size: (i + 1) * batch_size]
    y_gender_batch = y_gender_labels[i * batch_size: (i + 1) * batch_size]
    y_age_batch = y_age_labels[i * batch_size: (i + 1) * batch_size] 
#     s_length = train_sequence_length[i * batch_size: (i + 1) * batch_size]

    
    return (c_batch,a_batch, ad_batch, p_batch, in_batch, count_batch ,y_gender_batch,y_age_batch)  # ,a_batch in_batch,pc_batch, 

In [18]:
# plt.figure(1)
# sns.countplot(y_gender_labels)
# plt.figure(2)
# sns.countplot(y_age_labels)

In [19]:
device = torch.device('cuda:0')
class Model(nn.Module):
    def __init__(self,
                 gender_output_size=2, 
                 age_output_size=10,
                 count_input_size = 12,
                 hidden_size=32, 
                 sequence_length = 90,
                 vocab_size1=100, 
                 vocab_size2=100, 
                 vocab_size3=100, 
                 vocab_size4=100, 
                 vocab_size5=100,
                 embed_dim = 100,
                 bidirectional = True, use_pe = True,
                 dropout = 0.1, 
                 pre_trained=True, 
                 embeddings_weight_cid=None,
                 embeddings_weight_aid = None,
                 embeddings_weight_adid = None,
                 embeddings_weight_pid = None, 
                 embeddings_weight_in = None
                ):
        super(Model, self).__init__()
       
        self.gender_output_size = gender_output_size
        self.age_output_size = age_output_size

        self.count_input_size = count_input_size
        
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        
        self.embed_dim = embed_dim
        self.vocab_size1 = vocab_size1
        self.vocab_size2 = vocab_size2
        self.vocab_size3 = vocab_size3
        self.vocab_size4 = vocab_size4
        self.vocab_size5 = vocab_size5
        
        self.bidirectional = bidirectional
        self.use_pe = use_pe
        self.dropout = dropout
        self.pre_trained = pre_trained
        self._init_weights()
        
        if self.pre_trained:
            self.cid_embedding = nn.Embedding(self.vocab_size1, self.embed_dim, padding_idx=0).to(device)
            self.cid_embedding.weight.data = torch.FloatTensor(embeddings_weight_cid).to(device)
            self.cid_embedding.weight.requires_grad = False
            
            self.aid_embedding = nn.Embedding(self.vocab_size2, self.embed_dim, padding_idx=0).to(device)
            self.aid_embedding.weight.data = torch.FloatTensor(embeddings_weight_aid).to(device)
            self.aid_embedding.weight.requires_grad = False
            
            self.adid_embedding = nn.Embedding(self.vocab_size3, self.embed_dim, padding_idx=0).to(device)
            self.adid_embedding.weight.data = torch.FloatTensor(embeddings_weight_adid).to(device)
            self.adid_embedding.weight.requires_grad = False
            
            self.pid_embedding = nn.Embedding(self.vocab_size4, self.embed_dim, padding_idx=0).to(device)
            self.pid_embedding.weight.data = torch.FloatTensor(embeddings_weight_pid).to(device)
            self.pid_embedding.weight.requires_grad = False
            
            self.in_embedding = nn.Embedding(self.vocab_size5, self.embed_dim, padding_idx=0).to(device)
            self.in_embedding.weight.data = torch.FloatTensor(embeddings_weight_in).to(device)
            self.in_embedding.weight.requires_grad = False
        
        self.layer_size = 1
        self.kernel_size = 2
        
        self.out_channels = 128
        
        encoder_layer1 = nn.TransformerEncoderLayer(d_model = self.embed_dim, nhead = 4).to(device)
        self.encoder1 = nn.TransformerEncoder(encoder_layer1, 1).to(device)
        
        encoder_layer2 = nn.TransformerEncoderLayer(d_model = self.embed_dim, nhead = 4).to(device)
        self.encoder2 = nn.TransformerEncoder(encoder_layer2, 1).to(device)
        
        encoder_layer3 = nn.TransformerEncoderLayer(d_model = self.embed_dim, nhead = 4).to(device)
        self.encoder3 = nn.TransformerEncoder(encoder_layer3, 1).to(device)
        
        encoder_layer4 = nn.TransformerEncoderLayer(d_model = self.embed_dim, nhead = 4).to(device)
        self.encoder4 = nn.TransformerEncoder(encoder_layer4, 1).to(device)
        
        encoder_layer5 = nn.TransformerEncoderLayer(d_model = self.embed_dim, nhead = 4).to(device)
        self.encoder5 = nn.TransformerEncoder(encoder_layer5, 1).to(device)
        
        self.lstm1 = nn.GRU(self.embed_dim, self.hidden_size, self.layer_size,
                            bidirectional=self.bidirectional,batch_first = True,bias=True).to(device)

        self.lstm2 = nn.GRU(self.embed_dim, self.hidden_size, self.layer_size,
                            bidirectional=self.bidirectional,batch_first = True).to(device)
        
        self.lstm3 = nn.GRU(self.embed_dim, self.hidden_size, self.layer_size,
                            bidirectional=self.bidirectional,batch_first = True,bias=True).to(device)
        
        self.lstm4 = nn.GRU(self.embed_dim, self.hidden_size, self.layer_size,
                            bidirectional=self.bidirectional,batch_first = True,bias=True).to(device) 
        
        self.lstm5 = nn.GRU(self.embed_dim, self.hidden_size, self.layer_size,
                    bidirectional=self.bidirectional,batch_first = True,bias=True).to(device) 
        
        self.pool1 = nn.MaxPool1d(self.sequence_length).to(device)
        self.pool2 = nn.MaxPool1d(self.sequence_length).to(device)
        self.pool3 = nn.MaxPool1d(self.sequence_length).to(device)
        self.pool4 = nn.MaxPool1d(self.sequence_length).to(device)
        self.pool5 = nn.MaxPool1d(self.sequence_length).to(device)

        
        if self.bidirectional:
            self.layer_size = self.layer_size * 2
        
        self.shared_layer = nn.Sequential( 
                                nn.Linear(self.hidden_size * 5 + 512, 1024),
                                nn.BatchNorm1d(1024),
                                nn.ReLU(inplace = True),
                                nn.Dropout(0.5),
                                
                                nn.Linear(1024,512), 
                                nn.BatchNorm1d(512),
                                nn.ReLU(inplace=True),
                                nn.Dropout(0.5)).to(device)
        
        self.mlp = nn.Sequential( 
                                nn.Linear(self.count_input_size, 1024),
                                nn.BatchNorm1d(1024),
                                nn.ReLU(inplace = True),
                                nn.Dropout(0.5),
                                
                                nn.Linear(1024,512), 
                                nn.BatchNorm1d(512),
                                nn.ReLU(inplace=True),
                                nn.Dropout(0.5)).to(device)

        
        self.gender = nn.Linear(512, self.gender_output_size).to(device)
        
        self.age =  nn.Linear(512,self.age_output_size).to(device)
    
    def _init_weights(self):
        for p in self.parameters():
            if p.dim() > 1:
                kaiming_normal_(p, mode='fan_out', nonlinearity='relu')
   
    def get_padding_mask(self, batch_size, seq_len, inp_len):
        padding_mask = np.ones((batch_size, seq_len))
        for index, l in enumerate(inp_len):
            padding_mask[index,:l] = 0
        return torch.from_numpy(padding_mask).bool().to(device)
    
    def forward(self, x_cid, x_aid, x_adid,x_pid, x_in, x_count_feature):  #  x_pc, x_in,  x_aid
        
        x_c_f = self.mlp(x_count_feature)
        
        
#         batch_size, seq_len = x_cid.shape
#         inp_padding_mask = self.get_padding_mask(batch_size, seq_len, s_length)
        cid = self.cid_embedding(x_cid).permute(1,0,2)
        aid = self.aid_embedding(x_aid).permute(1,0,2)
        adid = self.adid_embedding(x_adid).permute(1,0,2)  # [sequnce_length, batch_size, embed_size]
        pid = self.pid_embedding(x_pid).permute(1,0,2)
        inid = self.in_embedding(x_in).permute(1,0,2)
        
        cout = self.encoder1(cid)  #  [sequnce_length, batch_size, embed_size]
        aout = self.encoder2(aid)
        adout = self.encoder3(adid)   # , src_key_padding_mask = inp_padding_mask
        pout = self.encoder4(pid)
        inout = self.encoder5(inid)
        
        cout = cout.permute(1,0,2) #  [batch_size,sequence_length,embed_size]
        aout = aout.permute(1,0,2)
        adout = adout.permute(1,0,2)
        pout = pout.permute(1,0,2)
        inout = inout.permute(1,0,2)
        
        cout = self.lstm1(cout)[0].permute(0,2,1)  #  batch_size,embed_size , equence_length,
        aout = self.lstm2(aout)[0].permute(0,2,1)
        adout = self.lstm3(adout)[0].permute(0,2,1)
        pout = self.lstm4(pout)[0].permute(0,2,1)
        inout = self.lstm5(inout)[0].permute(0,2,1)
        
        
        
        c_conv_out =  F.dropout(F.relu(self.pool1(cout)), p = 0.5, training = self.training)
        a_conv_out =  F.dropout(F.relu(self.pool2(aout)), p = 0.5, training = self.training)
        ad_conv_out = F.dropout(F.relu(self.pool3(adout)),p = 0.5, training = self.training)
        p_conv_out =  F.dropout(F.relu(self.pool4(pout)), p = 0.5, training = self.training)
        i_conv_out =  F.dropout(F.relu(self.pool5(inout)), p = 0.5, training = self.training)
#         conv_out = torch.cat((c_conv_out, a_conv_out,  ad_conv_out, pc_conv_out, in_conv_out),dim=1) # a_conv_out, 
#         out = conv_out.view(conv_out.shape[0] ,-1)
        out =  torch.cat((c_conv_out ,   a_conv_out , i_conv_out , ad_conv_out,p_conv_out ), dim = 1)# 
#         out = c_conv_out + ad_conv_out + p_conv_out
        out = out.squeeze()
        
        out = torch.cat((out, x_c_f), dim = 1)
        out = self.shared_layer(out)
        return self.gender(out),self.age(out)

    
model = Model(
            gender_output_size=2,
            age_output_size = 10,
            hidden_size=hidden_size,
            sequence_length = length_sequence,
            count_input_size = 12,
            embed_dim =100,
            
            vocab_size1=vocab_size1,
            vocab_size2=vocab_size2,
            vocab_size3=vocab_size3, 
            vocab_size4=vocab_size4,
            vocab_size5=vocab_size5,
            
            bidirectional=bidirectional,
            use_pe = use_pe,
            dropout=dropout,
            pre_trained=True,
            embeddings_weight_cid= word_embeddings_cid,
            embeddings_weight_aid= word_embeddings_aid,
            embeddings_weight_adid = word_embeddings_adid,
            embeddings_weight_pid = word_embeddings_pid, 
            embeddings_weight_in = word_embeddings_in, 
                    )
            

In [20]:
# # optimizer = RMSprop(model.parameters(),lr = lr, weight_decay = weight_decay)
encoder1_params = list(map(id,model.encoder1.parameters()))
encoder2_params = list(map(id,model.encoder2.parameters()))
encoder3_params = list(map(id,model.encoder3.parameters()))
encoder4_params = list(map(id,model.encoder4.parameters()))
encoder5_params = list(map(id,model.encoder5.parameters()))
base_params = filter(lambda p: id(p) not in encoder1_params + encoder2_params + encoder3_params + encoder4_params + encoder5_params, model.parameters())
optimizer = Adam([
    {'params':base_params},
    {'params':model.encoder1.parameters(), 'lr': lr / 2.0},
    {'params':model.encoder2.parameters(), 'lr': lr / 2.0},
    {'params':model.encoder3.parameters(), 'lr': lr / 2.0},
    {'params':model.encoder4.parameters(), 'lr': lr / 2.0},
    {'params':model.encoder5.parameters(), 'lr': lr / 2.0},
], lr = lr)
# optimizer = Adam(model.parameters(), lr = lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=0, threshold=1e-5, threshold_mode='abs')
    
lossfunc1 = torch.nn.CrossEntropyLoss()
lossfunc2 = torch.nn.CrossEntropyLoss()
# lossfunc2 = FocalLoss(class_num = 10)

In [21]:

# test_res_gender = []
# test_res_age = []

# def get_k_fold_data(k, i, X_cid, X_aid, X_adid, X_pid, X_in, Count_features, y_gender_labels, y_age_labels):
#     assert k > 1
#     fold_size = 900000 // k  # k = 5 3 input
    
#     X_cid_train, X_adid_train, y_gender_train, y_age_train = None, None, None, None
#     for j in range(k):
#         idx = slice(j * fold_size, (j+1) * fold_size)
#         x_cid_part, x_aid_part, x_adid_part, x_pid_part,x_in_part, count_features_part, y_gender_part, y_age_part = \
#         X_cid[idx,:],X_aid[idx,:], X_adid[idx,:], X_pid[idx,:],X_in[idx,:] ,Count_features[idx,:], y_gender_labels[idx],y_age_labels[idx]
#         if j == i :
#             X_cid_val,X_aid_val, X_adid_val, X_pid_val, X_in_val, Count_features_val, y_gender_val, y_age_val = \
#             x_cid_part, x_aid_part, x_adid_part, x_pid_part , x_in_part,count_features_part,  y_gender_part, y_age_part
#         elif X_cid_train is None:
#             X_cid_train, X_aid_train, X_adid_train, X_pid_train, X_in_train, Count_features_train, y_gender_train, y_age_train = \
#             x_cid_part, x_aid_part, x_adid_part, x_pid_part, x_in_part, count_features_part, y_gender_part, y_age_part
#         else:
#             X_cid_train = torch.cat((X_cid_train,x_cid_part),dim = 0)
#             X_aid_train = torch.cat((X_aid_train,x_aid_part),dim = 0)
#             X_adid_train = torch.cat((X_adid_train,x_adid_part),dim = 0)
#             X_pid_train = torch.cat((X_pid_train,x_pid_part),dim = 0)
#             X_in_train = torch.cat((X_in_train, x_in_part),dim = 0)
#             Count_features_train = torch.cat((Count_features_train,count_features_part),dim = 0)
#             y_gender_train = torch.cat((y_gender_train, y_gender_part),dim = 0)
#             y_age_train = torch.cat((y_age_train,y_age_part),dim = 0)
#     return X_cid_train, X_aid_train, X_adid_train, X_pid_train, X_in_train, Count_features_train, y_gender_train,y_age_train,\
#            X_cid_val, X_aid_val, X_adid_val,X_pid_val, X_in_val, Count_features_val, y_gender_val,y_age_val


# def k_fold(k, X_cid, X_aid, X_adid, X_pid, X_in, Count_features, y_gender_labels, y_age_labels):
    
#     train_loss_gender_sum = 0.0
#     valid_loss_gender_sum = 0.0

#     train_loss_age_sum = 0.0
#     valid_loss_age_sum = 0.0
    
    
    
#     train_acc_gender_sum = 0.0
#     valid_acc_gender_sum = 0.0

#     train_acc_age_sum = 0.0
#     valid_acc_age_sum = 0.0
#     val_res_gender = None
#     val_res_age = None

#     for i in range(k):
#         data = get_k_fold_data(k, i, X_cid, X_aid, X_adid, X_pid, X_in, count_feature, y_gender_labels, y_age_labels)
          
    
#         model = Model(
#                     gender_output_size=2,
#                     age_output_size = 10,
#                     hidden_size=hidden_size,
#                     sequence_length = length_sequence,
#                     count_input_size = 12,
#                     embed_dim =100,

#                     vocab_size1=vocab_size1,
#                     vocab_size2=vocab_size2,
#                     vocab_size3=vocab_size3,
#                     vocab_size4=vocab_size4,
#                     vocab_size5=vocab_size5,

#                     bidirectional=bidirectional,
#                     use_pe = use_pe,
#                     dropout=dropout,
#                     pre_trained=True,
#                     embeddings_weight_cid= word_embeddings_cid,
#                     embeddings_weight_aid= word_embeddings_aid,
#                     embeddings_weight_adid = word_embeddings_adid,
#                     embeddings_weight_pid = word_embeddings_pid, 
#                     embeddings_weight_in = word_embeddings_in, 
#                             )
            

#         model, val_gender_fold, val_age_fold , train_ls_gender, train_ls_age, valid_ls_gender, valid_ls_age  = train(i, model, *data)

#         if val_res_gender is None:
#             val_res_gender = val_gender_fold
#             val_res_age = val_age_fold
#         else:
#             val_res_gender = torch.cat((val_res_gender,val_gender_fold),0)
#             val_res_age = torch.cat((val_res_age,val_age_fold),0)


#         print('*'*25,'第',i+1,'折','*'*25)
#         print('train_gender_loss:%.4f'%train_ls_gender[-1][0],'train_gender_acc:%.4f\n'%train_ls_gender[-1][1],\
#               'train_age_loss:%.4f'%train_ls_age[-1][0],'train_age_acc:%.4f\n'%train_ls_age[-1][1],\
#               'valid_gender_loss:%.4f'%valid_ls_gender[-1][0],'valid_gender_acc:%.4f\n'%valid_ls_gender[-1][1],\
#               'valid_age_loss:%.4f'%valid_ls_age[-1][0],'valid_age_acc:%.4f\n'%valid_ls_age[-1][1]
#              )
#         train_loss_gender_sum += train_ls_gender[-1][0]
#         train_acc_gender_sum += train_ls_gender[-1][1]

#         train_loss_age_sum += train_ls_age[-1][0]
#         train_acc_age_sum += train_ls_age[-1][1]

#         valid_loss_gender_sum += valid_ls_gender[-1][0]
#         valid_acc_gender_sum += valid_ls_gender[-1][1]
#         valid_loss_age_sum += valid_ls_age[-1][0]
#         valid_acc_age_sum +=valid_ls_age[-1][1]
#         # record test output for stacking
#         _ = predict(model)
#         test_res_gender.append(_[0]) # average predict result
#         test_res_age.append(_[1])

#         del model 
#         _ = gc.collect()
#         torch.cuda.empty_cache()
#         time.sleep(5)
#     print('#'*10,'最终k折交叉验证结果','#'*10) 
#     print('train_loss_gender_sum:%.4f'%(train_loss_gender_sum/k),'train_acc_gender_sum:%.4f'%(train_acc_gender_sum/k),\
#           'train_loss_age_sum:%.4f'%( train_loss_age_sum/k),   'train_loss_age_sum:%.4f\n'%(train_acc_age_sum/k),\
#           'valid_loss_gender_sum:%.4f'%(valid_loss_gender_sum/k),'valid_acc_gender_sum:%.4f'%(valid_acc_gender_sum /k),\
#           'valid_loss_age_sum:%.4f'%(valid_loss_age_sum/k),'valid_acc_age_sum:%.4f'%(valid_acc_age_sum/k))
#     return val_res_gender, val_res_age
                                    

# def train(fold , model,X_cid_train, X_aid_train, X_adid_train, X_pid_train, X_in_train, Count_features_train, y_gender_train,y_age_train,\
#           X_cid_val, X_aid_val, X_adid_val,X_pid_val,X_in_val, Count_features_val, y_gender_val,y_age_val):
    
#     train_ls_gender, train_ls_age = [], []
#     valid_ls_gender, valid_ls_age = [], []
    
#     n = X_cid_train.shape[0]
#     m = X_cid_val.shape[0]
    
#     encoder1_params = list(map(id,model.encoder1.parameters()))
#     encoder2_params = list(map(id,model.encoder2.parameters()))
#     encoder3_params = list(map(id,model.encoder3.parameters()))
#     encoder4_params = list(map(id,model.encoder4.parameters()))
#     encoder5_params = list(map(id,model.encoder5.parameters()))
#     base_params = filter(lambda p: id(p) not in encoder1_params + encoder2_params + encoder3_params + encoder4_params + encoder5_params, model.parameters())
#     optimizer = Adam([
#         {'params':base_params},
#         {'params':model.encoder1.parameters(), 'lr': lr / 2.0},
#         {'params':model.encoder2.parameters(), 'lr': lr / 2.0},
#         {'params':model.encoder3.parameters(), 'lr': lr / 2.0},
#         {'params':model.encoder4.parameters(), 'lr': lr / 2.0},
#         {'params':model.encoder5.parameters(), 'lr': lr / 2.0},
#     ], lr = lr)

#     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=0, threshold=1e-5, threshold_mode='abs')
        
#     best_model = None
#     max_acc_sum = 0.0
#     val_gender_preds, val_age_preds = None, None
#     # training
    
#     for epoch in range(epochs):
#         print('epoch {}'.format(epoch+1))
#         model.train()
#         try:
#             with tqdm(range(n // batch_size)) as t:
#                 train_loss_gender = []
#                 train_loss_age = []
#                 train_acc_gender = 0.
#                 train_acc_age = 0.
#                 for i in t:
#                     c_batch,a_batch, ad_batch, p_batch, in_batch, count_batch ,y_gender_batch,y_age_batch = \
#                     get_batch(i, batch_size,X_cid_train, X_aid_train, X_adid_train, X_pid_train, X_in_train, Count_features_train, y_gender_train,y_age_train)
#                     optimizer.zero_grad()
#                     gender,age = model(c_batch.to(device),
#                                        a_batch.to(device), 
#                                        ad_batch.to(device), 
#                                        p_batch.to(device),
#                                        in_batch.to(device),
#                                        count_batch.to(device)
#                     )
                    
                    
#                     loss1 = lossfunc1(gender,y_gender_batch.to(device))     
#                     loss2 = lossfunc2(age,y_age_batch.to(device))

#                     t_loss = 0.5 * loss1  + 0.5 * loss2
                    
#                     train_loss_gender.append(float(loss1)) 
#                     train_loss_age.append(float(loss2))
#                     del loss1,loss2,c_batch, ad_batch

#                     pred_gender =  torch.max(gender,1)[1]
#                     pred_age =  torch.max(age,1)[1]

#                     train_correct_gender = (pred_gender.cpu() == y_gender_batch).sum()
#                     train_acc_gender += train_correct_gender.item()

#                     train_correct_age = (pred_age.cpu()== y_age_batch).sum()
#                     train_acc_age += train_correct_age.item()
        
                  
#                     t_loss.backward()
#                     nn.utils.clip_grad_norm_(model.parameters(), max_norm=100)
#                     optimizer.step()
# #                     scheduler.step()
                    
# #                     break
#                 print('Train_loss_gender:{:0.4f},acc_gender = {:0.4f}, Train_loss_age:{:0.4f},acc_age = {:0.4f}'.format(np.mean(train_loss_gender),train_acc_gender/n,np.mean(train_loss_age),train_acc_age/n))
#                 train_ls_gender.append((np.mean(train_loss_gender),train_acc_gender/ n) ) # plot train
#                 train_ls_age.append((np.mean(train_loss_age), train_acc_age/ n))
#         except KeyboardInterrupt:
#             t.close()
#             break
#         t.close()
#         # validation
#         torch.cuda.empty_cache()
#         model.eval()
#         eval_loss_gender = []
#         eval_loss_age = []
#         eval_acc_gender = 0.
#         eval_acc_age = 0.
#         _temp1, _temp2 = None, None
#         with torch.no_grad():
#             for i in range(m // batch_size):
                
#                 c_batch,a_batch, ad_batch, p_batch, in_batch, count_batch ,y_gender_batch,y_age_batch = \
#                 get_batch(i, batch_size,X_cid_val, X_aid_val, X_adid_val, X_pid_val, X_in_val,Count_features_val, y_gender_val,y_age_val)
#                 gender,age = model(c_batch.to(device),
#                                    a_batch.to(device), 
#                                    ad_batch.to(device), 
#                                    p_batch.to(device),
#                                    in_batch.to(device),
#                                    count_batch.to(device)
#                 )
#                 if _temp1 is None:
#                     _temp1 = gender
#                     _temp2 = age
#                 else:
#                     _temp1 = torch.cat((_temp1,gender), 0 )
#                     _temp2 = torch.cat((_temp2,age), 0 )
                    
#                 loss1 = lossfunc1(gender,y_gender_batch.to(device))
#                 loss2 = lossfunc2(age,y_age_batch.to(device))
#                 eval_loss_gender.append(loss1.item())
#                 eval_loss_age.append(loss2.item())
                
#                 pred_gender = torch.max(gender,1)[1]  # out_gender 
#                 eval_correct_gender = (pred_gender.cpu() == y_gender_batch).sum()
#                 eval_acc_gender += eval_correct_gender.item() 
                
#                 pred_age = torch.max(age,1)[1] # age_out
                
#                 eval_correct_age = ( pred_age.cpu() == y_age_batch).sum()
#                 eval_acc_age += eval_correct_age.item()

#             print('Eval_loss_gender:{:0.4f},acc_gender = {:0.4f}, Eval_loss_age:{:0.4f},acc_age = {:0.4f}'.format(np.mean(eval_loss_gender),eval_acc_gender / m, np.mean(eval_loss_age),eval_acc_age / m))
#             valid_ls_gender.append((np.mean(eval_loss_gender),eval_acc_gender/ m )) # plot train
#             valid_ls_age.append((np.mean(eval_loss_age), eval_acc_age/ m))
#             print('score = {:0.4f}'.format(eval_acc_gender / m  +  eval_acc_age / m))
#             scheduler.step(eval_acc_gender / m  +  eval_acc_age / m)
#             if eval_acc_gender / m  +  eval_acc_age / m > max_acc_sum:
#                 max_acc_sum = eval_acc_gender / m  +  eval_acc_age / m
#                 best_model = model
#                 val_gender_preds, val_age_preds = _temp1, _temp2
#     print('*'* 25, 'saving best model with max_acc_sum = ', max_acc_sum)
#     path = 'fold_'+str(fold) + 'best'+ save_path
#     torch.save(best_model.state_dict(),path)
#     return best_model,val_gender_preds, val_age_preds , train_ls_gender, train_ls_age, valid_ls_gender, valid_ls_age
    
# def predict(model):
#     model.eval()
#     test_prob_gender = None
#     test_prob_age = None
#     print('begin predict')
#     with torch.no_grad():
#         for i in tqdm(range( 1000000 // batch_size)):  # 10000 for test

#             c_id = torch.LongTensor(for_test_cid[i * batch_size : (i+1) * batch_size])
#             a_id = torch.LongTensor(for_test_aid[i * batch_size : (i+1) * batch_size])
#             ad_id = torch.LongTensor(for_test_adid[i * batch_size : (i+1) * batch_size])
#             p_id = torch.LongTensor(for_test_pid[i * batch_size : (i+1) * batch_size])
#             in_id = torch.LongTensor(for_test_in[i * batch_size : (i+1) * batch_size])
#             count_feature = torch.FloatTensor(for_test_count_feature[i * batch_size : (i+1) * batch_size])
# #             s_length = test_sequence_length[i * batch_size : (i+1) * batch_size]p_id.cuda(0), in_id.cuda(0),
#             gender,age = model(c_id.cuda(0),a_id.cuda(0), ad_id.cuda(0),p_id.cuda(0),in_id.cuda(0),count_feature.cuda(0))
#             if test_prob_gender is None:
#                 test_prob_gender = gender
#                 test_prob_age = age
#             else:
#                 test_prob_gender = torch.cat((test_prob_gender,gender),dim = 0)
#                 test_prob_age = torch.cat((test_prob_age,age),dim = 0)
#     return test_prob_gender,test_prob_age

In [22]:
# _gender, _age = k_fold(5, X_cid, X_aid, X_adid, X_pid, X_in, count_feature, y_gender_labels, y_age_labels)

In [23]:
# # # average predict_res
# preds_gender =  torch.mean(torch.stack(test_res_gender,0), 0)
# preds_age = torch.mean(torch.stack(test_res_age,0),0)
# preds_gender.shape,preds_age.shape


In [24]:
# train_set = np.hstack((_gender.cpu().detach().numpy() , _age.cpu().detach().numpy()))
# test_set = np.hstack((preds_gender.cpu().detach().numpy(),preds_age.cpu().detach().numpy()))
# train_set.shape,test_set.shape

In [25]:
# out_gender = np.zeros(1000000)
# out_age = np.zeros(1000000)
# out_gender = torch.max(preds_gender,1)[1].cpu().numpy() 
# out_age = torch.max(preds_age,1)[1].cpu().numpy()
# plt.figure(1)
# sns.countplot(out_gender)
# plt.figure(2)
# sns.countplot(out_age)

# output = pd.DataFrame(columns=['user_id'])
# # # print(len(test_preds_age))
# output['user_id'] = list(range(3000001,4000001))
# output['predicted_age'] = list(map(lambda x:int(x) + 1,out_age))
# output['predicted_gender']= list(map(lambda x:int(x) + 1,out_gender))

# import datetime
# date_p = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# output.to_csv('submit'+ date_p+ '.csv',index = False)


In [26]:
def train_with_early_stopping():
#     val_loss_full = []
    max_acc_sum = 0.
    for epoch in range(epochs):
        model.train()
        print('epoch {}'.format(epoch+1))
        train_loss_full = []
        val_loss_full = []
        try:
            with tqdm(range(890000 //batch_size)) as t:
                
                train_loss_gender = []
                train_loss_age = []
                train_acc_gender = 0.
                train_acc_age = 0.
                for i in t:
                    c_batch,a_batch, ad_batch, p_batch, in_batch, count_batch ,y_gender_batch,y_age_batch = \
                    get_batch(i,batch_size, X_cid,X_aid, X_adid,X_pid,X_in, count_feature, y_gender_labels, y_age_labels)
                    
                    optimizer.zero_grad()
                    gender,age = model(c_batch.to(device),
                                       a_batch.to(device), 
                                       ad_batch.to(device), 
                                       p_batch.to(device),
                                       in_batch.to(device),
                                       count_batch.to(device)
                    )
                    
                    loss1 = lossfunc1(gender,y_gender_batch.to(device))     
                    loss2 = lossfunc2(age,y_age_batch.to(device))

                    t_loss = 0.5 * loss1  +   0.5 *  loss2
                
                    
                    train_loss_gender.append(float(loss1)) 
                    train_loss_age.append(float(loss2))
                    del loss1,loss2,c_batch, ad_batch

                    pred_gender =  torch.max(gender,1)[1]
                    pred_age =  torch.max(age,1)[1]

                    train_correct_gender = (pred_gender.cpu() == y_gender_batch).sum()
                    train_acc_gender += train_correct_gender.item()

                    train_correct_age = (pred_age.cpu()== y_age_batch).sum()
                    train_acc_age += train_correct_age.item()
        
                  
                    t_loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), max_norm=100)
                    optimizer.step()
#                     scheduler.step()
                    
    
    

#                     break
                print('Train_loss_gender:{:0.4f},acc_gender = {:0.4f}, Train_loss_age:{:0.4f},acc_age = {:0.4f}'.format(np.mean(train_loss_gender),train_acc_gender/890000,np.mean(train_loss_age),train_acc_age/890000))
#                 train_loss_full.append(np.mean(train_loss)) # plot training loss
        except KeyboardInterrupt:
            t.close()
            break
        t.close()
        
        model.eval()
        eval_loss_gender = []
        eval_loss_age = []
        eval_acc_gender = 0.
        eval_acc_age = 0.
        # evaluation every_10_epoches
        with torch.no_grad():
            for i in range(890000 // batch_size, 900000//batch_size):
                
                c_batch,a_batch, ad_batch, p_batch, in_batch, count_batch ,y_gender_batch,y_age_batch = \
                get_batch(i,batch_size, X_cid,X_aid, X_adid,X_pid,X_in, count_feature, y_gender_labels, y_age_labels)    
                gender,age = model(c_batch.to(device),
                                       a_batch.to(device), 
                                       ad_batch.to(device), 
                                       p_batch.to(device),
                                       in_batch.to(device),
                                       count_batch.to(device)
                    )
                loss1 = lossfunc1(gender,y_gender_batch.to(device))
                loss2 = lossfunc2(age,y_age_batch.to(device))
                
                
                eval_loss_gender.append(loss1.item())
                eval_loss_age.append(loss2.item())
                
                pred_gender = torch.max(gender,1)[1]  # out_gender 
                eval_correct_gender = (pred_gender.cpu() == y_gender_batch).sum()
                eval_acc_gender += eval_correct_gender.item() 
                
                pred_age = torch.max(age,1)[1] # age_out
                
                eval_correct_age = ( pred_age.cpu() == y_age_batch).sum()
                eval_acc_age += eval_correct_age.item()

            print('Eval_loss_gender:{:0.4f},acc_gender = {:0.4f}, Eval_loss_age:{:0.4f},acc_age = {:0.4f}'.format(np.mean(eval_loss_gender),eval_acc_gender / 10000, np.mean(eval_loss_age),eval_acc_age / 10000))
            print('score = {:0.4f}'.format(eval_acc_gender /10000  +  eval_acc_age / 10000))
        scheduler.step(eval_acc_gender /10000 + eval_acc_age / 10000)
        # save best model
        if eval_acc_gender / 10000  +  eval_acc_age / 10000 > max_acc_sum:
            print('-' * 20 + 'saving best model at epoch ' + str(epoch + 1))
            max_acc_sum = eval_acc_gender / 10000 +  eval_acc_age / 10000
            torch.save(model,save_path)

            print('finished saving')
    print('training finished')

train_with_early_stopping()

  0%|          | 0/1780 [00:00<?, ?it/s]

epoch 1


100%|██████████| 1780/1780 [07:20<00:00,  4.04it/s]


Train_loss_gender:0.1852,acc_gender = 0.9343, Train_loss_age:1.4261,acc_age = 0.4123
Eval_loss_gender:0.1687,acc_gender = 0.9415, Eval_loss_age:1.3376,acc_age = 0.4449
score = 1.3864
--------------------saving best model at epoch 1


/home/lzp/.conda/envs/lzpven/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
  0%|          | 0/1780 [00:00<?, ?it/s]

...

Train_loss_gender:0.1494,acc_gender = 0.9490, Train_loss_age:1.2217,acc_age = 0.4913
Eval_loss_gender:0.1618,acc_gender = 0.9423, Eval_loss_age:1.2579,acc_age = 0.4809
score = 1.4232
--------------------saving best model at epoch 9


  0%|          | 0/1780 [00:00<?, ?it/s]

finished saving
epoch 10


100%|██████████| 1780/1780 [07:11<00:00,  4.12it/s]


Train_loss_gender:0.1475,acc_gender = 0.9496, Train_loss_age:1.2119,acc_age = 0.4950
Eval_loss_gender:0.1625,acc_gender = 0.9429, Eval_loss_age:1.2594,acc_age = 0.4801
score = 1.4230
training finished


In [27]:
torch.cuda.empty_cache()
model = torch.load(save_path,map_location = lambda storage,loc:storage.cuda(0))
print('loaded model')

loaded model


In [28]:
def test():
    with torch.no_grad():
        model.eval() 
        preds_gender = np.zeros(1000000)
        preds_age = np.zeros(1000000)
        print('begin predict')
        for i in tqdm(range( 1000000 // batch_size)):

            c_id = torch.LongTensor(for_test_cid[i * batch_size : (i+1) * batch_size])
            a_id = torch.LongTensor(for_test_aid[i * batch_size : (i+1) * batch_size])
            ad_id = torch.LongTensor(for_test_adid[i * batch_size : (i+1) * batch_size])
            p_id = torch.LongTensor(for_test_pid[i * batch_size : (i+1) * batch_size])
            in_id = torch.LongTensor(for_test_in[i * batch_size : (i+1) * batch_size])
            count_batch = torch.FloatTensor(for_test_count_feature[i * batch_size : (i+1) * batch_size])
#             s_length = test_sequence_length[i * batch_size : (i+1) * batch_size]
            gender,age = model(c_id.cuda(0),a_id.cuda(0),ad_id.cuda(0),p_id.cuda(0),in_id.cuda(0),count_batch.cuda(0))
            preds_gender[i * batch_size: (i+1) * batch_size] = torch.max(gender,1)[1].cpu().numpy() 
            preds_age[i * batch_size : (i+1) * batch_size] = torch.max(age,1)[1].cpu().numpy()
        plt.figure(1)
        sns.countplot(preds_gender)
        plt.figure(2)
        sns.countplot(preds_age)
        # output = test_embedding_features[['user_id']]

        output = pd.DataFrame(columns=['user_id'])
        # print(len(test_preds_age))
        output['user_id'] = list(range(3000001,4000001))
        output['predicted_age'] = list(map(lambda x:int(x) + 1,preds_age))
        output['predicted_gender']= list(map(lambda x:int(x) + 1,preds_gender))

        import datetime
        date_p = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        output.to_csv('submit'+ date_p+ '.csv',index = False)

    
test()

  0%|          | 0/2000 [00:00<?, ?it/s]

begin predict


TypeError: expected Long (got Float)

In [ ]:
# def get_outputs(count_feature):
#     torch.cuda.empty_cache()
#     model.eval()
    
#     X_train_gender, X_train_age = None, None
#     X_val_gender, X_val_age = None, None
#     X_test_gender, X_test_age = None, None
#     with torch.no_grad():
#         for i in tqdm(range(720000 // batch_size)):
            
#             c_batch,a_batch, ad_batch, p_batch, in_batch, count_batch ,y_gender_batch,y_age_batch = \
#             get_batch(i,batch_size, X_cid,X_aid, X_adid,X_pid,X_in, count_feature, y_gender_labels, y_age_labels)
#             gender,age = model(c_batch.cuda(1),a_batch.cuda(1),ad_batch.cuda(1),p_batch.cuda(1), in_batch.cuda(1), count_batch.cuda(1))

#             gender_prob, age_prob = gender.cpu().detach().numpy() ,age.cpu().detach().numpy()

#             if X_train_gender is None:
#                 X_train_gender = gender_prob
#                 X_train_age = age_prob
#             else:
#                 X_train_gender = np.vstack((X_train_gender, gender_prob))
#                 X_train_age = np.vstack((X_train_age, age_prob))
#         print(X_train_gender.shape,X_train_age.shape)

#         for i in range(720000 // batch_size, 900000 // batch_size):
#             c_batch,a_batch, ad_batch, p_batch, in_batch, count_batch ,y_gender_batch,y_age_batch = \
#             get_batch(i,batch_size, X_cid,X_aid, X_adid,X_pid,X_in, count_feature, y_gender_labels, y_age_labels)
            
#             gender,age = model(c_batch.cuda(1),a_batch.cuda(1),ad_batch.cuda(1),p_batch.cuda(1), in_batch.cuda(1), count_batch.cuda(1))


#             gender_prob, age_prob = gender.cpu().detach().numpy(), age.cpu().detach().numpy()
#             if X_val_gender is None:
#                 X_val_gender = gender_prob
#                 X_val_age = age_prob
#             else:
#                 X_val_gender = np.vstack((X_val_gender, gender_prob))
#                 X_val_age = np.vstack((X_val_age, age_prob))
#         print(X_val_gender.shape,X_val_age.shape)

#         for i in tqdm(range(1000000 // batch_size)):


#             c_id = torch.LongTensor(for_test_cid[i * batch_size : (i+1) * batch_size])
#             a_id = torch.LongTensor(for_test_aid[i * batch_size : (i+1) * batch_size])
#             ad_id = torch.LongTensor(for_test_adid[i * batch_size : (i+1) * batch_size])
#             p_id = torch.LongTensor(for_test_pid[i * batch_size : (i+1) * batch_size])
#             in_id = torch.LongTensor(for_test_in[i * batch_size : (i+1) * batch_size])
#             count_feature = torch.FloatTensor(for_test_count_feature[i * batch_size : (i+1) * batch_size])
# #             s_length = test_sequence_length[i * batch_size : (i+1) * batch_size]
#             gender,age = model(c_id.cuda(1),a_id.cuda(1),ad_id.cuda(1),p_id.cuda(1),in_id.cuda(1),count_feature.cuda(1))
#             gender_prob, age_prob = gender.cpu().detach().numpy(),age.cpu().detach().numpy()
#             if X_test_gender is None:
#                 X_test_gender = gender_prob
#                 X_test_age = age_prob
#             else:
#                 X_test_gender = np.vstack((X_test_gender, gender_prob))
#                 X_test_age = np.vstack((X_test_age, age_prob))
#     print(X_test_gender.shape,X_test_age.shape)
    
#     return X_train_gender,X_train_age,X_val_gender,X_val_age,X_test_gender,X_test_age

        
# # X_train_gender,X_train_age,X_val_gender,X_val_age,X_test_gender,X_test_age = get_outputs(count_feature)     

In [ ]:
# X_train = np.hstack((X_train_gender,X_train_age))
# X_val = np.hstack((X_val_gender,X_val_age))
# X_test = np.hstack((X_test_gender,X_test_age))
# X_train.shape

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier
# lr_model1 = LogisticRegression(C = 1)
# lr_model1.fit(train_set, y_gender_labels.numpy())
# r2_score = lr_model1.score(train_set[720000:900000],y_gender_labels[720000:900000].numpy())
# print(r2_score)
# gender_preds = lr_model1.predict(test_set)
# sns.countplot(gender_preds)

In [ ]:
# lr_model2 = LogisticRegression(C = 1)
# lr_model2.fit(train_set, y_age_labels.numpy())
# r2_score = lr_model2.score(train_set[720000:900000],y_age_labels[720000:900000].numpy())
# print(r2_score)
# age_preds = lr_model2.predict(test_set)
# sns.countplot(age_preds)

In [ ]:
# output = pd.DataFrame(columns=['user_id'])
# # print(len(test_preds_age))
# output['user_id'] = list(range(3000001,4000001))
# output['predicted_age'] = list(map(lambda x:int(x) + 1,age_preds))
# output['predicted_gender']= list(map(lambda x:int(x) + 1,gender_preds))

# import datetime
# date_p = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# output.to_csv('submit'+ date_p+ '.csv',index = False)

_________________________TEST___________________________________________________________________

In [ ]:
# # 输出性别

# # test_x = scaler.transform(test_embedding_features.iloc[:,1:129])
# # print(test_x)

# test = test_user
# # scaler2 = StandardScaler()
# # scaler2.fit(test_1)
# # test = scaler2.transform(test_1)
# test_preds = lbt1.predict(test) # xgboost
# test_preds =[round(v) + 1 for v in test_preds]
# # print(test_preds)
# sns.countplot(test_preds)
# # test_acc = accuracy_score(y_test,test_preds)
# # print('test_acc = ', test_acc)

In [ ]:
# # 输出年龄
# test_preds_age = lbt2.predict(test) # lightgbm
# test_preds_age =[round(v) + 1 for v in test_preds_age]
# # print(test_preds_age)
# sns.countplot(test_preds_age)
# # test_acc = accuracy_score(y_test,test_preds)
# # print('test_acc = ', test_acc)

In [ ]:
#         self.c_conv = nn.Sequential(
#             nn.Conv1d(in_channels = self.hidden_size * 2  + self.embed_dim1,
#                       out_channels = self.out_channels,
#                       kernel_size = self.kernel_size),
#             nn.ReLU(inplace = True),
#             nn.BatchNorm1d(self.out_channels),
            
# #             nn.Conv1d(in_channels = self.out_channels,
# #                       out_channels = self.out_channels,
# #                       kernel_size = self.kernel_size),
# #             nn.ReLU(inplace = True),
# #             nn.BatchNorm1d(self.out_channels),
           
#                                     ).to(device)
        
#         self.a_conv = nn.Sequential(
#             nn.Conv1d(in_channels = self.hidden_size * 2 + self.embed_dim2,
#                       out_channels = self.out_channels,
#                       kernel_size = self.kernel_size),
#             nn.ReLU(inplace = True),
#             nn.BatchNorm1d(self.out_channels),
            
# #             nn.Conv1d(in_channels = self.out_channels,
# #                       out_channels = self.out_channels,
# #                       kernel_size = self.kernel_size),
# #             nn.ReLU(inplace = True),
# #             nn.BatchNorm1d(self.out_channels),
            
#                                 ).to(device)
        
#         self.ad_conv = nn.Sequential(
#             nn.Conv1d(in_channels = self.hidden_size * 2+ self.embed_dim3,
#                       out_channels = self.out_channels,
#                       kernel_size = self.kernel_size),
#             nn.ReLU(inplace = True),
#             nn.BatchNorm1d(self.out_channels),
            
                            
# #             nn.Conv1d(in_channels = self.out_channels,
# #                       out_channels = self.out_channels,
# #                       kernel_size = self.kernel_size),
# #             nn.ReLU(inplace = True),
# #             nn.BatchNorm1d(self.out_channels),
            
#                             ).to(device)
        
        
#         self.pc_conv = nn.Sequential(
#             nn.Conv1d(in_channels = self.hidden_size * 2 + self.embed_dim4,
#                       out_channels = self.out_channels,
#                       kernel_size = self.kernel_size),
#             nn.ReLU(inplace = True),
#             nn.BatchNorm1d(self.out_channels),
            
                            
# #             nn.Conv1d(in_channels = self.out_channels,
# #                       out_channels = self.out_channels,
# #                       kernel_size = self.kernel_size),
# #             nn.ReLU(inplace = True),
# #             nn.BatchNorm1d(self.out_channels),
            
#                             ).to(device)
        
#         self.in_conv = nn.Sequential(
#             nn.Conv1d(in_channels = self.hidden_size * 2 + self.embed_dim5,
#                       out_channels = self.out_channels,
#                       kernel_size = self.kernel_size),
#             nn.ReLU(inplace = True),
#             nn.BatchNorm1d(self.out_channels),
            
                            
# #             nn.Conv1d(in_channels = self.out_channels,
# #                       out_channels = self.out_channels,
# #                       kernel_size = self.kernel_size),
# #             nn.ReLU(inplace = True),
# #             nn.BatchNorm1d(self.out_channels),

In [ ]:
# class_num =10
# def search_weight(valid_y, raw_prob, init_weight=[1.0]*class_num, step=0.001):
#     weight = init_weight.copy()
#     f_best = accuracy_score(y_true=valid_y, y_pred=raw_prob.argmax(axis=1))
#     flag_score = 0
#     round_num = 1
#     while(flag_score != f_best):
#         print("round: ", round_num)
#         round_num += 1
#         flag_score = f_best
#         for c in range(class_num):
#             for n_w in range(0, 2000,10):
#                 num = n_w * step
#                 new_weight = weight.copy()
#                 new_weight[c] = num
#                 prob_df = raw_prob.copy()

#                 prob_df = prob_df * np.array(new_weight)
#                 f = accuracy_score(y_true=valid_y, y_pred=prob_df.argmax(

#                     axis=1))
#                 if f > f_best:
#                     weight = new_weight.copy()
#                     f_best = f
#                     print(f)
#     return weight
#     class_num = 10
    
#     w = np.zeros((10000 // batch_size, class_num))

#     # search weight
#     for i in range(890000 // batch_size, 900000//batch_size):
        
#         c_batch,a_batch,ad_batch, ct_mask , y_gender_batch,y_age_batch = get_batch(i,batch_size)
# #                 c_tf_batch, a_tf_batch ,ad_tf_batch, pc_tf_batch, in_tf_batch = get_tfidf_features(i,batch_size)
#         ct_mask = ct_mask.unsqueeze(2).repeat(1,1, 2 * hidden_size)
# #                 ct_mask = (ct_mask / torch.sum(ct_mask,dim=1).unsqueeze(1)).unsqueeze(2).repeat(1,1,2 * hidden_size)
#         gender,age = model(c_batch.cuda(2),
#                            a_batch.cuda(2), 
#                            ad_batch.cuda(2), 
# #                            pc_batch.cuda(1),
# #                            in_batch.cuda(1), 
#                            ct_mask.cuda(2))
#         del c_batch, a_batch, ad_batch
#         weight = search_weight(y_age_batch.detach().numpy(), F.softmax(age,dim=1).cpu().detach().numpy())
#         w[i - 890000 // batch_size] = weight
    
    
#     weights = np.mean(w, axis = 0) # numpy
#     print(weights)